In [122]:
import cv2 as cv2
import numpy as np
%pylab inline
import matplotlib.pyplot as plt
import itertools as itertools 
import time
import os
import pandas as pd
import scipy.spatial as scipy_spatial
from skimage import color
import pickle
import sklearn
import cyvlfeat as vlfeat
import math
from scipy.spatial.distance import cdist
from sklearn.svm import LinearSVC, SVC
from utils import *
import random
from shutil import copyfile
import ntpath

Populating the interactive namespace from numpy and matplotlib


d:\1920sem1\cs4243\cs4243-lab3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random', 'shuffle']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [33]:
img_save_dir = 'D:\\1920Sem1\\CS4243\\project_wx_tryout_code\\rectangles'

image_dir = "D:\\1920Sem1\CS4243\project\CS4243-Project\datasets\JPEGImages"
images = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, img))]

In [34]:
def preprocess_input_img(img_0):
    hsv = cv2.cvtColor(img_0, cv2.COLOR_RGB2HSV)

    lower_red = np.array([110,80,80])
    upper_red = np.array([160,255,255])
    mask_red_only = cv2.inRange(hsv, lower_red, upper_red)

    lower_white = np.array([0,0,255])
    upper_white = np.array([255,255,255])
    mask_white_only = cv2.inRange(hsv, lower_white, upper_white)
    
    # attach a column of row number to the masks
    index_array = np.flip(np.arange(mask_red_only.shape[1]))

    stacked_red = np.vstack((index_array, mask_red_only))
    stacked_red_white = np.vstack((stacked_red, mask_white_only))

    red_mask_rot = np.rot90(stacked_red)
    red_white_mask_rot = np.rot90(stacked_red_white)
    
    return red_white_mask_rot

In [35]:
def get_suspected_waldo_stripe_region_for_col(red_white_mask_col, ratio_range=(1/2, 2), white_pixel_threshold=0.15):
    col_index = red_white_mask_col[0]
    red_mask_col = red_white_mask_col[1:int((len(red_white_mask_col)-1)/2+1)]
    white_mask_col = red_white_mask_col[int((len(red_white_mask_col)-1)/2+1):]
    assert red_mask_col.shape == white_mask_col.shape, "the mask column inputs for white and red masks are of different shapes:" + str(red_mask_col.shape) + "; " + str(white_mask_col.shape)
    
    
    white_mask_col = white_mask_col[1:]
    
    new_red_mask_col = np.zeros(len(red_mask_col))
    new_red_mask_col[red_mask_col == 255] = 1
    new_red_mask_col[red_mask_col == 0] = -1
    
    new_white_mask_col = np.zeros(len(white_mask_col))
    new_white_mask_col[white_mask_col == 255] = 1
    new_white_mask_col[white_mask_col == 0] = 0

    streak_array = get_streak_len_array_with_approx(new_red_mask_col, 0)
    ratio_array = np.array([abs(streak_array[i]) / abs(streak_array[i + 1]) for i in range(len(streak_array) - 1)])
    accepted_ratio_indice, = np.where(np.logical_and(ratio_array > ratio_range[0], ratio_array < ratio_range[1]))
    is_accepted_ratio_indice_continuous, lst_of_start_and_end = is_indice_continuous(accepted_ratio_indice)

    revised_lst_of_start_and_end = []

    if is_accepted_ratio_indice_continuous:
        # then check if the negative pixels are white pixels
        for start, end in lst_of_start_and_end:
            is_region_start_with_red = streak_array[start+2] > 0
            offset = 0
            if is_region_start_with_red:
                offset = 1
            
            total_number_of_white_pixels = 0
            total_number_of_non_red_pixels = 0
#             region = [np.sum(np.abs(streak_array[:accepted_ratio_indice[start]])), np.sum(np.abs(streak_array[:accepted_ratio_indice[end-1]+1]))]
#             print("start, end: " + str(region[0]) + ", " + str(region[1]))

            while start + offset < end:
                supposedly_white_pixel_region = [np.sum(np.abs(streak_array[:accepted_ratio_indice[start + offset]])), np.sum(np.abs(streak_array[:accepted_ratio_indice[start + offset]+1]))]

                number_of_white_pixels = np.sum(new_white_mask_col[int(supposedly_white_pixel_region[0]) : int(supposedly_white_pixel_region[1])])
                number_of_non_red_pixels = supposedly_white_pixel_region[1] - supposedly_white_pixel_region[0]

                total_number_of_white_pixels += number_of_white_pixels
                total_number_of_non_red_pixels += number_of_non_red_pixels
                offset += 2

            if total_number_of_white_pixels / total_number_of_non_red_pixels > white_pixel_threshold:
                revised_lst_of_start_and_end.append((start, end))
        
        if len(revised_lst_of_start_and_end) == 0:
            empty_terms = np.array(list(itertools.repeat([int(False), col_index, -1, -1, 0, 0], 20)))
            return empty_terms
        stripe_region = [[int(True), \
                          col_index, \
                          np.sum(np.abs(streak_array[:accepted_ratio_indice[start]])), \
                          np.sum(np.abs(streak_array[:accepted_ratio_indice[end-1]+1])), \
                          int(end - start), \
                          (np.sum(np.abs(streak_array[:accepted_ratio_indice[end-1]+1])) - np.sum(np.abs(streak_array[:accepted_ratio_indice[start]]))) / int(end - start)] \
                         for start, end in revised_lst_of_start_and_end]
        # from here examine the percentage of white pixels in the region
        # if there are white pixels dominating the negative pixels, then it is stripes
        # return (1, col_number, start_of_stripe_region, end_of_stripe_region, number_of_stripes, average_width_of_stripes)        
        how_many_more_terms = 20 - len(stripe_region)
        additional_terms = np.array(list(itertools.repeat([int(False), col_index, -1, -1, 0, 0], how_many_more_terms)))
        return np.vstack((stripe_region, additional_terms))    

    empty_terms = np.array(list(itertools.repeat([int(False), col_index, -1, -1, 0, 0], 20)))
    return empty_terms

In [36]:
def get_streak_len_array_with_approx(unit_lst, threshold):
    # in the unit_lst, 1 represents the superior pixel value, -1 is the inferior pixel value
    # the superior pixel value is the one that is well captured in color distillation 
    # in our case e.g. red is the superior pixel value because it is well captured 
    # and white is the inferior pixel value because it might be noise
    # we want to restore the shape outlined by the superior pixel value
    # hence we need to minimize the disturbannce by the inferior pixel value
    # hence we need to remove the 'sparse' inferior pixel value located within the streaks of superior pixel values
    # the definition of sparse is given by the threshold
    previous = 0
    streak_len_array = []
    streak_len = 0
    fast_forward_count = 0
    for i in range(len(unit_lst)):
        if i == 0:
            previous = unit_lst[i]
            
        if fast_forward_count > 0:
            fast_forward_count -= 1
            if i == len(unit_lst) - 1:
                streak_len_array.append(streak_len)
            continue
        
        if previous == unit_lst[i]:
            cur_streak_len = streak_len
            streak_len += unit_lst[i]
            if i == len(unit_lst) - 1:
                streak_len_array.append(streak_len)
            previous = unit_lst[i]
        else:
            if streak_len > 0:
                # superior pixel streak ending
                cur_streak_len = streak_len
                # we need to look ahead the threshold number of pixels to see if theres any superior pixel
                next_few_number = min(threshold, len(unit_lst) - 1 - i)
                next_few_pixels = unit_lst[i : i+1+next_few_number]
                # if there is we will continue the superior pixel streak from there
                indice_of_next_superior_pixel, = np.where(next_few_pixels == 1)
                if len(indice_of_next_superior_pixel) != 0:
                    next_superior = max(indice_of_next_superior_pixel)
                    streak_len += 1 + next_superior
                    fast_forward_count = next_superior
                    previous = abs(unit_lst[i])
                # if not we end the superior pixel streak here and start a new inferior pixel streak
                else:
                    streak_len_array.append(cur_streak_len)
                    streak_len = 0
                    streak_len += unit_lst[i]
                    previous = unit_lst[i]
            elif streak_len < 0:
                # inferior pixel streak ending
                # we will start a new superior pixel streak
                streak_len_array.append(streak_len)
                streak_len = 0
                streak_len += unit_lst[i]
                previous = unit_lst[i]
            
    return streak_len_array

def get_streak_len_array(unit_lst):
    previous = 0
    streak_len_array = []
    streak_len = 0
    for i in range(len(unit_lst)):
        if i == 0:
            previous = unit_lst[i]
        
        if previous == unit_lst[i]:
            streak_len += unit_lst[i]
            if i == len(unit_lst) - 1:
                streak_len_array.append(streak_len)
        else:
            streak_len_array.append(streak_len)
            streak_len = 0
            
        previous = unit_lst[i]
        
    return streak_len_array


# Method: Given a 1D array, find the starting position and the ending position of an arithmatic sequence with the next equal to 1+previous. The sequence must contain no less than 3 numbers.
# Input: a 1D array of integers
# Output: a tuple in the form of (Boolean, list). The boolean is True when there is at least one sequence. The list consists of tuples (starting_position_of_sequence_inclusive, ending_position_of_sequence_exclusive)
def is_indice_continuous(lst_of_indice):
    # right now the number of stripes accepted is 3
    is_continuous = False
    continuous_count = 0
    prev = -1
    start_index = -1
    lst_of_start_and_end = []
    for i in range(len(lst_of_indice)):
        if i == 0:
            prev = lst_of_indice[i]
            start_index = i
            continuous_count = 1
            continue
            
        if lst_of_indice[i] - prev == 1:
            # the streak continues
            continuous_count += 1
        else:
            # the streak ends
            if continuous_count >= 3:
                lst_of_start_and_end.append((start_index, i))
                is_continuous = True

            continuous_count = 1
            start_index = i
            
        if i == len(lst_of_indice) - 1:
            if start_index != -1 and continuous_count >= 3:
                lst_of_start_and_end.append((start_index, i))
                is_continuous = True
        
        prev = lst_of_indice[i]
        
    return (is_continuous, lst_of_start_and_end)

In [6]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

def convert_rot90_col_no_to_original_col_no(rot90_image, col_no):
    return rot90_image.shape[0] - col_no

def convert_original_col_no_to_rot90_col_no(image, col_no):
    return image.shape[1] - col_no

In [7]:
def is_able_to_combine(row1, row2, confidence_area_overlap_ratio_threshold):
    row1_width = row1[3] - row1[2]
    row1_height = row1[5] - row1[4]
    row2_width = row2[3] - row2[2]
    row2_height = row2[5] - row2[4]
    
    bbox1 = (row1[2], row1[4], row1_width, row1_height)
    bbox2 = (row2[2], row2[4], row2_width, row2_height)
    
    iou = IoU(bbox1, bbox2)
    
    if (iou >= confidence_area_overlap_ratio_threshold):
        return True
    else:
        return False
    
def IoU(bbox1, bbox2):
    """ Compute IoU of two bounding boxes

    Args:
        bbox1 - 4-tuple (x, y, w, h) where (x, y) is the top left corner of
            the bounding box, and (w, h) are width and height of the box.
        bbox2 - 4-tuple (x, y, w, h) where (x, y) is the top left corner of
            the bounding box, and (w, h) are width and height of the box.
    Returns:
        score - IoU score
    """
    x1, y1, w1, h1 = bbox1
    x2, y2, w2, h2 = bbox2
    score = 0
    ### YOUR CODE HERE
    
    x_left = max(x1, x2)
    y_top = max(y1, y2)
    x_right = min(x1+w1, x2+w2)
    y_bottom = min(y1+h1, y2+h2)

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    area1 = w1 * h1
    area2 = w2 * h2
    score = intersection_area / float(area1 + area2 - intersection_area)
    ### END YOUR CODE

    return score

    
def merge_rows(data, row_1_index, row_2_index):
#     ['col_no_original', 'stripe_centre_for_each_col',
#        'col_confidence_interval_start', 'col_confidence_interval_end',
#        'row_confidence_interval_start', 'row_confidence_interval_end']
    row_1 = data[row_1_index][:-1]
    row_2 = data[row_2_index][:-1]
    row_1_no_of_times_combined = data[row_1_index][-1]
    row_2_no_of_times_combined = data[row_2_index][-1]
    total_no_of_times_combined = row_2_no_of_times_combined + row_1_no_of_times_combined
    row_3 = np.add(row_1*row_1_no_of_times_combined, row_2*row_2_no_of_times_combined) / total_no_of_times_combined
    
    row_3 = np.append(row_3, total_no_of_times_combined)
#     new_data = np.append(data, row_3.reshape((1, len(row_3))), axis=0)
#     new_data = np.delete(new_data, [row_1_index, row_2_index], axis=0)
    data[row_1_index] = row_3
    new_data = np.delete(data, [row_2_index], axis=0)
    return new_data

def combine_to_region(df, confidence_area_overlap_ratio_threshold):
    if ("no_of_times_combined" not in df.columns):
        df["no_of_times_combined"] = 1
        
    all_data = df[:].to_numpy()
    
    data = np.copy(all_data)
    
    count = 0
    
    while (True):
        if count >= len(data) - 1:
            return pd.DataFrame(data=data, columns=df.columns)
        else:
            this_row = data[count]
            rest_rows = np.delete(data, count, axis=0)
            this_vector = this_row[:2]
            rest_vectors = rest_rows[:, :2]
            # find the vector with the shortest distance
            dist = scipy_spatial.distance.cdist(this_vector.reshape((1,2)), rest_vectors)
            closest_vector_index = np.argmin(dist)
            closest_row = rest_rows[closest_vector_index]
            # if can combine, then merge, and the index will take the one of the smaller index, recurse.
            can_combine = is_able_to_combine(this_row, closest_row, confidence_area_overlap_ratio_threshold)
            if can_combine:
                # merge, and the index will take the one of the smaller index, recurse.
                if count <= closest_vector_index:
                    closest_vector_index += 1
                new_data = merge_rows(data, count, closest_vector_index)
                count = 0
                data = new_data
            else:
                # count++, recurse.
                count += 1


In [8]:
def get_raw_stripe_confidence_regions_df_from_suspected_stripe_regions(result):
    col_confidence_interval_width = 8
    row_confidence_interval_width = 10
    reshaped = np.reshape(result, (result.shape[0] * result.shape[1], result.shape[2]))
    df = pd.DataFrame(data=reshaped, columns=["has_stripes", "col_no", "stripe_start", "stripe_end", "no_of_stripes", "average_stripe_width"])
    df = df[df["has_stripes"]==1]
    df = df[df["average_stripe_width"]>=2]
    tidy_df = pd.DataFrame()
    tidy_df["col_no_original"] = convert_rot90_col_no_to_original_col_no(np.rot90(img_0), df["col_no"])
    tidy_df["stripe_centre_for_each_col"] = (df["stripe_start"] + df["stripe_end"]) / 2

    tidy_df["col_confidence_interval_start"] = tidy_df["col_no_original"] - df["average_stripe_width"] * col_confidence_interval_width / 2
    tidy_df["col_confidence_interval_start"] = np.where(tidy_df["col_confidence_interval_start"]< 0, 0, tidy_df["col_confidence_interval_start"]) 

    tidy_df["col_confidence_interval_end"] = tidy_df["col_no_original"] + df["average_stripe_width"] * col_confidence_interval_width / 2
    tidy_df["col_confidence_interval_end"] = np.where(tidy_df["col_confidence_interval_end"]>img_0.shape[1]-1, img_0.shape[1]-1, tidy_df["col_confidence_interval_end"]) 

    tidy_df["row_confidence_interval_start"] = tidy_df["stripe_centre_for_each_col"] - df["average_stripe_width"] * row_confidence_interval_width / 2
    tidy_df["row_confidence_interval_start"] = np.where(tidy_df["row_confidence_interval_start"]< 0, 0, tidy_df["row_confidence_interval_start"]) 

    tidy_df["row_confidence_interval_end"] = tidy_df["stripe_centre_for_each_col"] + df["average_stripe_width"] * row_confidence_interval_width / 2
    tidy_df["row_confidence_interval_end"] = np.where(tidy_df["row_confidence_interval_end"]>img_0.shape[0]-1, img_0.shape[0]-1, tidy_df["row_confidence_interval_end"]) 
    
    return tidy_df

def get_combined_stripe_confidence_regions_df_from_raw_stripe_confidence_regions_df(raw_df, combine_threshold=0.2):
    return combine_to_region(raw_df, combine_threshold)

In [9]:
def draw_rectangles_from_stripe_region_df(df, img, img_name):
    df_for_rectangles = df[['row_confidence_interval_start', 'col_confidence_interval_start', 'row_confidence_interval_end', 'col_confidence_interval_end']]
    rectangles = df_for_rectangles[:].to_numpy().astype(int)
    img_copy = np.copy(img)
    count = 0
    for rect in rectangles:
        cv2.rectangle(img_copy, (rect[1], rect[0]), (rect[3], rect[2]), color=(0, 255, 0), thickness=16)
        count += 1
        
    img_full_name = img_name + "_" + str(count) + ".jpeg"
    cv2.imwrite(os.path.join(img_save_dir, img_full_name), img_copy)

In [38]:
range_square_save_dir = 'D:\\1920Sem1\\CS4243\\project_wx_tryout_code\\range_square'
range_save_dir = 'D:\\1920Sem1\\CS4243\\project_wx_tryout_code\\range'

def match_wanda_face(template_original, range_to_match, kernel_size, range_name):
    template = cv2.resize(template_original, (kernel_size, kernel_size))
    
    w, h = template.shape[::-1]
    res = cv2.matchTemplate(range_to_match, template, cv2.TM_CCOEFF_NORMED)
    res_max_value = res[np.unravel_index(np.argmax(res, axis=None), res.shape)]
#     print(kernel_size)
#     print(range_name + ": " + str(res_max_value))
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)
    
    range_to_match_copy = range_to_match.copy()
    
    square_cropped = range_to_match_copy[top_left[1]:top_left[1] + h , top_left[0]:top_left[0] + w ]
    imgsquare_name = "square" + "_" + range_name + "_" + str(int(res_max_value*100)) + ".png"
    cv2.imwrite(os.path.join(range_square_save_dir, imgsquare_name), square_cropped)

#     cv2.rectangle(range_to_match_copy,top_left, bottom_right, 255, 2)   
    imgname = range_name + "_" + str(int(res_max_value*100)) + ".png"
    cv2.imwrite(os.path.join(range_save_dir, imgname), range_to_match_copy)
    
#     print(square_cropped.shape)
    return square_cropped

In [43]:
def all_files_in_dir(dir):
    return [os.path.join(dir, img) for img in os.listdir(dir) if os.path.isfile(os.path.join(dir, img))]

In [69]:
range_square_save_dir = 'D:\\1920Sem1\\CS4243\\project_wx_tryout_code\\range_square'
range_save_dir = 'D:\\1920Sem1\\CS4243\\project_wx_tryout_code\\range'

template_wenda_dir = 'D:\\1920Sem1\\CS4243\\project_wx_tryout_code\\range_square\\wenda'
def match_wanda_face_dir(template_original_dir, range_to_match, kernel_size, range_name):
    template_originals = all_files_in_dir(template_original_dir)[:3]
    
    range_to_match_copy = range_to_match.copy()
    range_to_match_clean = range_to_match.copy()
    
    bboxes = []
    squares_cropped = []
    res_values = []
    for template_original_address in template_originals:
        template_original = cv2.imread(template_original_address, 0)
        w = template_original.shape[0]
        
        template = cv2.resize(template_original, (kernel_size, kernel_size))

        w, h = template.shape[::-1]
        res = cv2.matchTemplate(range_to_match, template, cv2.TM_CCOEFF_NORMED)
        res_max_value = res[np.unravel_index(np.argmax(res, axis=None), res.shape)]
    #     print(kernel_size)
    #     print(range_name + ": " + str(res_max_value))
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        top_left = max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)
        
        bbox = (top_left[0], top_left[1], w, h)
        bboxes.append(bbox)
        
        square_cropped = range_to_match_clean[top_left[1]:top_left[1] + h , top_left[0]:top_left[0] + w ]
        squares_cropped.append(square_cropped)
        res_values.append(res_max_value)

        cv2.rectangle(range_to_match_copy,top_left, bottom_right, 255, 2)   
    
    iou_3 = IoU_3(bboxes[0], bboxes[1], bboxes[2])
    
    if (iou_3 > -0.2):
#         ind = res_values.index(max(res_values))
        for k in range(3):
            square_cropped = squares_cropped[k]
            imgsquare_name = "square" + str(k) + "_" + range_name + "_" + str(int(res_max_value*100)) + ".png"
            cv2.imwrite(os.path.join(range_square_save_dir, imgsquare_name), square_cropped)
        
        imgname = range_name + "_" + str(int(res_max_value*100)) + ".png"
        cv2.imwrite(os.path.join(range_save_dir, imgname), range_to_match_copy)

    #     print(square_cropped.shape)
#         return square_cropped

In [ ]:


    #     print(square_cropped.shape)
#         return square_cropped

In [70]:
def IoU_3(bbox1, bbox2, bbox3):
    """ Compute IoU of two bounding boxes

    Args:
        bbox1 - 4-tuple (x, y, w, h) where (x, y) is the top left corner of
            the bounding box, and (w, h) are width and height of the box.
        bbox2 - 4-tuple (x, y, w, h) where (x, y) is the top left corner of
            the bounding box, and (w, h) are width and height of the box.
        bbox3 - 4-tuple (x, y, w, h) where (x, y) is the top left corner of
            the bounding box, and (w, h) are width and height of the box.
    Returns:
        score - IoU score
    """
    x1, y1, w1, h1 = bbox1
    x2, y2, w2, h2 = bbox2
    x3, y3, w3, h3 = bbox3
    score = 0
    ### YOUR CODE HERE
    
    y1y2_x_left = max(x1, x2)
    y1y2_y_top = max(y1, y2)
    y1y2_x_right = min(x1+w1, x2+w2)
    y1y2_y_bottom = min(y1+h1, y2+h2)
    
    if y1y2_x_right < y1y2_x_left or y1y2_y_bottom < y1y2_y_top:
        return 0.0
    
    y1y3_x_left = max(x1, x3)
    y1y3_y_top = max(y1, y3)
    y1y3_x_right = min(x1+w1, x3+w3)
    y1y3_y_bottom = min(y1+h1, y3+h3)
    
    if y1y3_x_right < y1y3_x_left or y1y3_y_bottom < y1y3_y_top:
        return 0.0
    
    y3y2_x_left = max(x3, x2)
    y3y2_y_top = max(y3, y2)
    y3y2_x_right = min(x3+w3, x2+w2)
    y3y2_y_bottom = min(y3+h3, y2+h2)

    if y3y2_x_right < y3y2_x_left or y3y2_y_bottom < y3y2_y_top:
        return 0.0
  
    y1y2y3_x_left = max(x1, x2, y3)
    y1y2y3_y_top = max(y1, y2, y3)
    y1y2y3_x_right = min(x1+w1, x2+w2, x3+w3)
    y1y2y3_y_bottom = min(y1+h1, y2+h2, y3+h3)

    if y1y2y3_x_right < y1y2y3_x_left or y1y2y3_y_bottom < y1y2y3_y_top:
        return 0.0
    
    y1y2_intersection_area = (y1y2_x_right - y1y2_x_left) * (y1y2_y_bottom - y1y2_y_top)
    y1y3_intersection_area = (y1y3_x_right - y1y3_x_left) * (y1y3_y_bottom - y1y3_y_top)
    y3y2_intersection_area = (y3y2_x_right - y3y2_x_left) * (y3y2_y_bottom - y3y2_y_top)

    y1y2y3_intersection_area = (y1y2y3_x_right - y1y2y3_x_left) * (y1y2y3_y_bottom - y1y2y3_y_top)
    
    y1_area1 = w1 * h1
    y2_area2 = w2 * h2
    y3_area3 = w3 * h3

    total_area = float(y1_area1 + y2_area2 + y3_area3 - y1y2_intersection_area - y1y3_intersection_area - y3y2_intersection_area + y1y2y3_intersection_area)
    score = y1y2y3_intersection_area / total_area
    ### END YOUR CODE

    return score

In [12]:
def get_suspected_waldo_stripe_region_for_img(img):

    red_white_mask_rot = preprocess_input_img(img)

    start_time = time.time()
    result  = np.apply_along_axis(get_suspected_waldo_stripe_region_for_col, 1, red_white_mask_rot)
    stop_time = time.time()
    print("--- %s seconds ---" % (stop_time - start_time))
    
    return result

In [103]:
def match_face(template_original_dir, img, row, kernel_size, range_name, square_cropped_save_dir):
    
    row_start, col_start, row_end, col_end, stripe_width = row
    
    range1 = img[row_start:row_end, col_start:col_end]
    range_to_match = cv2.cvtColor(range1, cv2.COLOR_BGR2GRAY)
    
    template_originals = all_files_in_dir(template_original_dir)[:3]
    
    range_to_match_copy = range_to_match.copy()
    range_to_match_clean = range_to_match.copy()
    
    bboxes = []
    squares_cropped = []
#     res_values = []
    for template_original_address in template_originals:
        template_original = cv2.imread(template_original_address, 0)
        w = template_original.shape[0]
        
        template = cv2.resize(template_original, (kernel_size, kernel_size))

        w, h = template.shape[::-1]
        res = cv2.matchTemplate(range_to_match, template, cv2.TM_CCOEFF_NORMED)
        res_max_value = res[np.unravel_index(np.argmax(res, axis=None), res.shape)]
    #     print(kernel_size)
    #     print(range_name + ": " + str(res_max_value))
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        top_left = max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)
        
        bbox = (col_start + top_left[0], row_start + top_left[1], w, h)
        bboxes.append(bbox)
        
        square_cropped = range_to_match_clean[top_left[1]:top_left[1] + h , top_left[0]:top_left[0] + w ]
        squares_cropped.append(square_cropped)
#         res_values.append(res_max_value)

#         cv2.rectangle(range_to_match_copy,top_left, bottom_right, 255, 2)   
        
    for k in range(3):
        square_cropped = squares_cropped[k]
        imgsquare_name = "template" + str(k) + "_" + range_name + ".png"
        cv2.imwrite(os.path.join(square_cropped_save_dir, imgsquare_name), square_cropped)

#     imgname = range_name + "_" + str(int(res_max_value*100)) + ".png"
#     cv2.imwrite(os.path.join(range_save_dir, imgname), range_to_match_copy)
    
    return bboxes



In [105]:
def build_vocabulary(image_paths, vocab_size):
    """
      This function will sample SIFT descriptors from the training images,
      cluster them with kmeans, and then return the cluster centers.

      Args:
      -   image_paths: list of image paths.
      -   vocab_size: size of vocabulary

      Returns:
      -   vocab: This is a vocab_size x d numpy array (vocabulary). Each row is a
          cluster center / visual word
    """
    # Load images from the training set. To save computation time, you don't
    # necessarily need to sample from all images, although it would be better
    # to do so. You can randomly sample the descriptors from each image to save
    # memory and speed up the clustering. Or you can simply call vl_dsift with
    # a large step size here, but a smaller step size in get_bags_of_sifts.
    #
    # For each loaded image, get some SIFT features. You don't have to get as
    # many SIFT features as you will in get_bags_of_sift, because you're only
    # trying to get a representative sample here. You can try taking 20 features
    # per image.
    #
    # Once you have tens of thousands of SIFT features from many training
    # images, cluster them with kmeans. The resulting centroids are now your
    # visual word vocabulary.

    dim = 128      # length of the SIFT descriptors that you are going to compute.
    vocab = np.zeros((vocab_size,dim))
    total_SIFT_features = np.zeros((20*len(image_paths), dim))

    #############################################################################
    # TODO: YOUR CODE HERE                                                      #
    #############################################################################

    # raise NotImplementedError('`build_vocabulary` function needs to be implemented')
    descriptor_collection = np.zeros((1, dim))
    sample_img_paths = image_paths
#     sample_img_paths = np.random.choice(image_paths, int(len(image_paths)/10))

    for i in range(len(sample_img_paths)):
    # for i in range(10):
        img = cv2.imread(sample_img_paths[i], 0)
        N = 50
        # N = 4
        step = max(int((img.shape[0] / N)), 1)
        size = 4
             
        frames, descriptors = vlfeat.sift.dsift(img, step=step, size=size)
        # print(descriptors.shape)
        # descriptors = descriptors[:21, :]
        descriptor_collection = np.vstack((descriptor_collection, descriptors))
        
    trimmed_descriptor_collection = descriptor_collection[1:,:]
    cluster_centers = vlfeat.kmeans.kmeans(trimmed_descriptor_collection, vocab_size)
    vocab = cluster_centers
    #############################################################################
    #                             END OF YOUR CODE                              #
    #############################################################################

    return vocab


In [106]:
def bags_of_sifts(image_paths, vocab_filename):
    """
      You will want to construct SIFT features here in the same way you
      did in build_vocabulary() (except for possibly changing the sampling
      rate) and then assign each local feature to its nearest cluster center
      and build a histogram indicating how many times each cluster was used.
      Don't forget to normalize the histogram, or else a larger image with more
      SIFT features will look very different from a smaller version of the same
      image.

      Args:
      -   image_paths: paths to N images
      -   vocab_filename: Path to the precomputed vocabulary.
              This function assumes that vocab_filename exists and contains an
              vocab_size x 128 ndarray 'vocab' where each row is a kmeans centroid
              or visual word. This ndarray is saved to disk rather than passed in
              as a parameter to avoid recomputing the vocabulary every run.

      Returns:
      -   image_feats: N x d matrix, where d is the dimensionality of the
              feature representation. In this case, d will equal the number of
              clusters or equivalently the number of entries in each image's
              histogram (vocab_size) below.
    """
    # load vocabulary
    with open(vocab_filename, 'rb') as f:
        vocab = pickle.load(f)

    # dummy features variable
    feats = []

    #############################################################################
    # TODO: YOUR CODE HERE                                                      #
    #############################################################################

    # raise NotImplementedError('`get_bags_of_sifts` function needs to be implemented')
    for i in range(len(image_paths)):
    # for i in range(10):
        img = cv2.imread(image_paths[i], 0)
        N = 50
        # N = 4
        step = max(int((img.shape[0] / N)), 1)
        size = 4
             
        frames, descriptors = vlfeat.sift.dsift(img, step=step, size=size)
        D = cdist(descriptors, vocab)
        indice_of_closest_vocab = np.argmin(D, axis=1)
        histogram = np.zeros(vocab.shape[0])
        for ind in indice_of_closest_vocab:
            histogram[ind] += 1
        normalized_histogram = normalize(histogram)
        feats.append(normalized_histogram)
        
    feats = np.array(feats)
    #############################################################################
    #                             END OF YOUR CODE                              #
    #############################################################################

    return feats

def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v / norm

In [118]:
def svm_classify(train_image_feats, train_labels, test_image_feats):
    """
    This function will train a one-versus-one support vector machine (SVM)
    and then use the learned classifiers to predict the category of every test image. 

    Args:
    -   train_image_feats:  N x d numpy array, where d is the dimensionality of
            the feature representation
    -   train_labels: N element list, where each entry is a string indicating the
            ground truth category for each training image
    -   test_image_feats: M x d numpy array, where d is the dimensionality of the
            feature representation. You can assume N = M, unless you have changed
            the starter code
    Returns:
    -   test_labels: M element list, where each entry is a string indicating the
            predicted category for each testing image
    """
    categories = list(set(train_labels))
    test_labels = []

    #############################################################################
    # TODO: YOUR CODE HERE                                                      #
    #############################################################################
    
    # raise NotImplementedError('`svm_classify` function needs to be implemented')
    svm = SVC(C=1000)
    model = svm.fit(train_image_feats, train_labels)
    test_labels = model.predict(test_image_feats)
    #############################################################################
    #                             END OF YOUR CODE                              #
    #############################################################################

    return test_labels

In [120]:
def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

In [72]:
# for i in range(len(images)-5, len(images)-3):
#     print("image " + str(i) + ":") 

#     img_0 = cv2.imread(images[i])

#     result = get_suspected_waldo_stripe_region_for_img(img_0)
#     raw_stripe_confidence_regions_df = get_raw_stripe_confidence_regions_df_from_suspected_stripe_regions(result)
#     combined_stripe_confidence_regions_df = get_combined_stripe_confidence_regions_df_from_raw_stripe_confidence_regions_df(raw_stripe_confidence_regions_df)

#     draw_rectangles_from_stripe_region_df(combined_stripe_confidence_regions_df, img_0, "img_"+str(i))

image 65:
--- 63.08299899101257 seconds ---
image 66:
--- 1.81504225730896 seconds ---


In [133]:
# Below are everything for one image

SyntaxError: unexpected character after line continuation character (<ipython-input-133-b651db5fc0f6>, line 1)

In [73]:
num = 65
img_0 = cv2.imread(images[num])

range_save_dir = 'D:\\1920Sem1\\CS4243\\project_wx_tryout_code\\range'
range_square_save_dir = 'D:\\1920Sem1\\CS4243\\project_wx_tryout_code\\range_square'
template_wanda = cv2.imread('template_wanda_2.png', 0)

red_white_mask_rot = preprocess_input_img(img_0)

print("image " + str(num) + ":") 

result = get_suspected_waldo_stripe_region_for_img(img_0)
raw_stripe_confidence_regions_df = get_raw_stripe_confidence_regions_df_from_suspected_stripe_regions(result)
combined_stripe_confidence_regions_df = get_combined_stripe_confidence_regions_df_from_raw_stripe_confidence_regions_df(raw_stripe_confidence_regions_df)

df_for_rectangles = combined_stripe_confidence_regions_df.copy()

df_for_head = df_for_rectangles[['row_confidence_interval_start', 'col_confidence_interval_start', 'row_confidence_interval_end', 'col_confidence_interval_end']].copy()
df_for_head['stripe_width'] = (df_for_head['col_confidence_interval_end'] - df_for_head['col_confidence_interval_start'])/8

# to expand the body bbox horizontally a bit and extend the body bbox upwards to bound the head
df_for_head['col_confidence_interval_start'] = np.where(df_for_head['col_confidence_interval_start'] - 2*df_for_head['stripe_width']<0, 0, df_for_head['col_confidence_interval_start'] - 2*df_for_head['stripe_width'])
df_for_head['col_confidence_interval_end'] = np.where(df_for_head['col_confidence_interval_end'] + 2*df_for_head['stripe_width']>img_0.shape[1]-1, img_0.shape[1]-1, df_for_head['col_confidence_interval_end'] + 2*df_for_head['stripe_width'])
df_for_head['row_confidence_interval_start'] = np.where(df_for_head['row_confidence_interval_start'] - 10*df_for_head['stripe_width']<0, 0, df_for_head['row_confidence_interval_start'] - 10*df_for_head['stripe_width'])
df_for_head['row_confidence_interval_end'] = np.where(df_for_head['row_confidence_interval_end'] + 1*df_for_head['stripe_width']>img_0.shape[0]-1, img_0.shape[0]-1, df_for_head['row_confidence_interval_end'] + 1*df_for_head['stripe_width'])





--- 79.69399905204773 seconds ---
--- 0.7400422096252441 seconds ---
(48, 7)


In [79]:
range_data = df_for_head[:].to_numpy()

range_data_stripe_centre = df_for_rectangles[['col_no_original', 'stripe_centre_for_each_col']]

In [75]:
for i in range(range_data.shape[0]):

    wanda_row = tuple(range_data[i].astype(int))
    wanda_row_start, wanda_col_start, wanda_row_end, wanda_col_end, wanda_stripe_width = wanda_row

    range1 = img_0[wanda_row_start:wanda_row_end, wanda_col_start:wanda_col_end]
    range1_gray = cv2.cvtColor(range1, cv2.COLOR_BGR2GRAY)

    head_stripe_counts = [3, 5, 6]
    for head_stripe_count in head_stripe_counts:
        kernel_size = (head_stripe_count*wanda_stripe_width // 2) * 2 + 1

        match_wanda_face_dir(template_wenda_dir, range1_gray, kernel_size, str(head_stripe_count) + "_" + str(num) + "_range"+str(i))

In [127]:
square_cropped_save_dir = 'square_cropped_save_dir'
identified_square_save_dir = 'identified_square_save_dir'

In [ ]:
for f in all_files_in_dir(square_cropped_save_dir):
    os.remove(f)

In [101]:
bboxes = []
for i in range(range_data.shape[0]):

    wanda_row = tuple(range_data[i].astype(int))
    wanda_row_start, wanda_col_start, wanda_row_end, wanda_col_end, wanda_stripe_width = wanda_row

    range1 = img_0[wanda_row_start:wanda_row_end, wanda_col_start:wanda_col_end]
    range1_gray = cv2.cvtColor(range1, cv2.COLOR_BGR2GRAY)

    head_stripe_counts = [3, 5, 6]
    
    bboxes_of_row = []
    
    for j in range(len(head_stripe_counts)):
        head_stripe_count = head_stripe_counts[j]
        kernel_size = (head_stripe_count*wanda_stripe_width // 2) * 2 + 1
        bboxes_of_three_templates = match_face(template_wenda_dir, img_0, wanda_row, kernel_size, "countind" + str(j) + "_" + "img" +  str(num) + "_range"+str(i), square_cropped_save_dir)
        bboxes_of_row.append(bboxes_of_three_templates)
        
    
    bboxes.append(bboxes_of_row)  

In [154]:
def get_row_no_from_square_name(square_file_name):
    square_file_name = square_file_name[:-4]
    split_info = square_file_name.split("_")
    row_no = split_info[3][split_info[3].index('range') + len('range'):]
    return int(row_no)

def bbox_of_identified_square(square_file_name, bboxes):
    square_file_name = square_file_name[:-4]
    split_info = square_file_name.split("_")
    template_no = split_info[0][split_info[0].index('template') + len('template'):]
    countind_no = split_info[1][split_info[1].index('countind') + len('countind'):]
    row_no = split_info[3][split_info[3].index('range') + len('range'):]
    
    return bboxes[int(row_no)][int(countind_no)][int(template_no)]

In [110]:
wenda_square_save_dir = 'range_square\\wenda'
wenda_waldo_square_save_dir = 'range_square\\wenda_waldo'
rubbish_square_save_dir = 'range_square\\rubbish'

wenda_squares = all_files_in_dir(wenda_square_save_dir)
wenda_waldo_squares = all_files_in_dir(wenda_waldo_square_save_dir)
rubbish_squares = all_files_in_dir(rubbish_square_save_dir)

test_squares = all_files_in_dir(square_cropped_save_dir)

In [111]:
vocab_filename = 'vocab.pkl'
if not os.path.isfile(vocab_filename):
    print('No existing visual word vocabulary found. Computing one from training images')
    vocab_size = 100  # Larger values will work better (to a point) but be slower to compute
    vocab = build_vocabulary(wenda_squares, vocab_size)
    
    with open(vocab_filename, 'wb') as f:
        pickle.dump(vocab, f)
        print('{:s} saved'.format(vocab_filename))

In [114]:
random_index = [i for i in range(400)]
if (len(rubbish_squares) > 400):
    random_index = random.sample(range(len(rubbish_squares)), 400)
    
train_imgs = wenda_squares + [rubbish_squares[i] for i in random_index] 
train_labels = ['wenda' for i in range(len(wenda_squares))] + ["rubbish" for i in random_index] 


In [115]:
train_feats = bags_of_sifts(train_imgs, 'vocab.pkl')
test_feats = bags_of_sifts(test_squares, 'vocab.pkl')

In [119]:
test_labels = svm_classify(train_feats, train_labels, test_feats)
detected_wendas = [test_squares[i] for i in range(len(test_labels)) if test_labels[i]=='wenda']

square_cropped_save_dir\template0_countind0_img65_range36.png


d:\1920sem1\cs4243\cs4243-lab3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [128]:
for f in all_files_in_dir(identified_square_save_dir):
    os.remove(f)
    
for detected_wenda in detected_wendas:
    copyfile(detected_wenda, os.path.join(identified_square_save_dir, path_leaf(detected_wenda)))


In [129]:
for f in all_files_in_dir(square_cropped_save_dir):
    os.remove(f)

for f in all_files_in_dir(identified_square_save_dir):
    copyfile(f, os.path.join(square_cropped_save_dir, path_leaf(f)))

In [130]:
train_imgs = wenda_squares + wenda_waldo_squares 
train_labels = ['wenda' for i in range(len(wenda_squares))] + ["waldo" for i in range(len(wenda_waldo_squares))] 
test_squares = all_files_in_dir(square_cropped_save_dir)

In [131]:
train_feats = bags_of_sifts(train_imgs, 'vocab.pkl')
test_feats = bags_of_sifts(test_squares, 'vocab.pkl')

In [132]:
test_labels = svm_classify(train_feats, train_labels, test_feats)
detected_wendas = [test_squares[i] for i in range(len(test_labels)) if test_labels[i]=='wenda']
for f in all_files_in_dir(identified_square_save_dir):
    os.remove(f)
    
for detected_wenda in detected_wendas:
    copyfile(detected_wenda, os.path.join(identified_square_save_dir, path_leaf(detected_wenda)))

d:\1920sem1\cs4243\cs4243-lab3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [151]:
path_leaf(detected_wenda)

'template1_countind0_img65_range36.png'

In [155]:
for f in all_files_in_dir(identified_square_save_dir):
    bbox_of_square = bbox_of_identified_square(path_leaf(f), bboxes)
    row_no_of_square = get_row_no_from_square_name(path_leaf(f))
    # bbox_of_range: [row_start, col_start, row_end, col_end, stripe_width]
    bbox_of_range = range_data[row_no_of_square]
    # centre_of_stripes_in_range: ['col_no_original', 'stripe_centre_for_each_col']
    centre_of_stripes_in_range = range_data_stripe_centre[:].to_numpy()[row_no_of_square]
    print(bbox_of_square)
    print(bbox_of_range)
    print(centre_of_stripes_in_range)

(1438, 4430, 51, 51)
[4282.14285714 1391.28571429 4648.14285714 1600.42857143   17.42857143]
[1495.85714286 4543.57142857]
(6590, 5980, 25, 25)
[5.9035e+03 6.5830e+03 6.0085e+03 6.6430e+03 5.0000e+00]
[6613.  5978.5]
(1444, 4432, 51, 51)
[4282.14285714 1391.28571429 4648.14285714 1600.42857143   17.42857143]
[1495.85714286 4543.57142857]
